In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import os

import scipy.stats as stats
import warnings

from matplotlib import pyplot as plt
from dotenv import load_dotenv
from sklearn.decomposition import PCA
from sklearn import preprocessing

from scripts.pca_fn import show_contribution, display_circles
from scripts.cleaning import remove_outliers, scale_df

load_dotenv()
plt.style.use('Solarize_Light2')
sns.color_palette("colorblind")

warnings.filterwarnings('ignore')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

pc_dpi = int(os.getenv('DPI'))

if pc_dpi is None:
    pc_dpi = 100


In [ ]:
# File

sea_ds = "./data/seattle_data.csv"

df_seattle = pd.read_csv(sea_ds)


In [ ]:
df_seattle.shape

# <u>Analyse exploratoire :</u>

### <u>0. Elimination d'aberrations via IQR</u>

### <u>1. Age des batiments </u>
- <u>1.a : Représentation des batiments en fonction de leurs dates de construction. </u>
- <u>1.b : Visualisation des statistiques principales de la variable : Energie par metre carré </u>
- <u>1.c : Paire 1 : Date de construction et utilisation en energie par metre carré. </u>

<hr>

### <u> 2. Etude de la note Energy Star</u>
- <u> 2.a : Repartition de la note Energy Star au sein des données. </u>
- <u> 2.b : Paire 2 : Note Energy Star en fonction de l'energie par metre carré. </u>
- <u> 2.c : Paire 3 : Energy Star en fonction de l'année de construction. </u>
- <u> 2.d : Paire 4 : Energy Star en fonction de l'émission de gazs a effets de serre. </u>
<hr>


### <u> 3 : Représentation Property Type/Utilisation Energie </u>
- <u> 3.a : Visualisation des categories principales </u>
- <u> 3.b : Etude en détails de la correllation taille -> E* </u>

### <u> 4 : Reduction de dimension via ACP </u>
- <u> 4.a : Elimination des variables inutiles la modelisation </u>
- <u> 4.b : Realisation d'une ACP sur les variables restantes </u>

### Cette analyse permettra de :
 - Visualiser "l'age" du paysage de Seattle
 - Contextualiser la mesure de la consommation d'énergie au metre carré, et sa possible relation avec l'age des batiments
 - Avoir une meilleure comprehension de la variable Energy Star et de la présenter face à l'age des batiments et de leur consommation en energie


## 0. Elimination d'outliers via la methode interquartile :

In [ ]:
cleaning_list = ["Electricity(kWh)", "SourceEUI(kWh/m2)", "SiteEnergyUse(kWh)"]

for col in cleaning_list:
    remove_outliers(col, df_seattle)

print(df_seattle.shape)

print(f"{1099 - df_seattle.shape[0]} buildings deleted")


## 1. Age des batiments

### 1.a : Représentation des batiments en fonction de leurs dates de construction.

In [ ]:
data_age = list(df_seattle["YearBuilt"].astype(int))
oldest = min(data_age)
youngest = max(data_age)

print(f"Le plus ancien batiment date de {oldest}, le plus récent (donnees de 2016) date de {youngest}")


In [ ]:
decades = np.arange(1900, 2020, 10)

building_per_decade = dict.fromkeys(decades)

for decade in building_per_decade.keys():
    built = len([year for year in data_age if (year > decade and year < decade + 10)])
    building_per_decade[decade] = built

used_cmap = plt.get_cmap("Dark2")

fig, (ax1, ax2) = plt.subplots(
    ncols=2,
    nrows=1,
    figsize=(22, 10),
    gridspec_kw={"width_ratios": [1, 4]},
    dpi=pc_dpi
    )

ax1.boxplot(data_age, showmeans=True, widths=0.25)

ax2.bar(building_per_decade.keys(), height=building_per_decade.values(), width=8, color=used_cmap.colors)
# ax2 lineplot

###
# Titles & Labels
ax1.set_ylabel("Annee de construction")
ax2.set_ylabel("Nombre de batiments construits")
ax2.set_xlabel("Annee de construction")
fig.suptitle("Representation des batiments en fonction de leur annee de construction")
fig.tight_layout()
#
###

plt.show()


#### Observations :

- Le paysage urbanin de Seattle date, majoritairement (q1, q3) de 1939 - 1990 (moyenne en 1965)
- On remarque une logique économique : pic de construction dans les années 20 (Roaring 20s) suivi par un creux dans les années 30 (Grande Depression)
- De nombreux projets date de du debut du millénaire, la decenie 2010+ n'est pas étudiable : les données s'arretent en 2015

### 1.b : Visualisation des statistiques principales de la variable : Energie par metre carré

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    ncols=2,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
    )

source_eui_data = list(df_seattle["SourceEUI(kWh/m2)"].values.astype(float))

sns.boxplot(data=source_eui_data, width=0.6, ax=ax1, color="navy")
ax1.set_ylim(0, 1400)  # High number of outliers, cf describe
sns.boxplot(data=source_eui_data, width=0.6, ax=ax2, color="red")
# fig.tight_layout()

###
# Titles/Lables
ax1.set_ylabel("Consommation d'energie (kWh/m2)")
fig.suptitle("Visualisation de la repartition de l'EUI parmi les données (energy use intensity)\
 gauche : zoom sur 0-1400kW/m2")
#
###

plt.show()

print(df_seattle["SourceEUI(kWh/m2)"].astype(float).describe())


#### Analyse :

- 1 : Les outliers sont extrèmement nombreux, uniquement dans les utilisations "hautes"
- 2 : La majorité des batiments ont une EUI entre 215 et 494 kWh/m2, 369 en moyenne.
- 3 : L'élimination de batiments via la methode interquartile peut être viable, la reduction de dimension via ACP peut également être une piste


### 1.c : Paire 1 : Date de construction et utilisation en energie par metre carré

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
)

ax1 = sns.barplot(
    x=df_seattle["YearBuilt"].astype(int).values,
    y=df_seattle["SourceEUI(kWh/m2)"].values.astype(float),
    ci=5
)

###
# Titles/Lables
ax1.set_xticklabels([])
ax1.set_ylabel("SourceEUI(kWh/m2)")
ax1.set_xlabel("Année de construction (1900 - 2015)")
fig.suptitle("Visualisation de 'energy use intensity' moyenne en fonction de l'année de construction des batiments")

#
###
fig.tight_layout()
plt.show()


#### Observation : 
- L'age des batiments ne semble pas avoir d'impact sur leur intensité d'utilisation énergétique. On peut emettre l'hypothèse que les batiments les plus anciens ont été modernisés pour avoir une consommation energetique plus responsable.

## 2. Etude de la note Energy Star

### 2.a : Repartition de la note Energy Star au sein des données

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    ncols=2,
    nrows=1,
    figsize=(22, 12),
    gridspec_kw={"width_ratios": [1, 4]},
    dpi=pc_dpi,
)

rated = df_seattle[df_seattle["ENERGYSTARScore"].notna()]
scores = list(rated["ENERGYSTARScore"].astype(int).values)
score_range = range(0, 101, 1)
scores_dict = dict.fromkeys(score_range)

for score in scores_dict.keys():
    scores_dict[score] = scores.count(score)

ax1.boxplot(scores, showmeans=True, widths=0.25)

ax2.bar(
    x=list(scores_dict.keys()),
    height=list(scores_dict.values()),
    color=used_cmap.colors,
    width=1
    )

####
## Titles/Lables
ax1.set_ylabel("Nombre de batiments")
ax1.set_xlabel("Note Energy Star (0-100)")
fig.suptitle("Répartition de la note Energy Star au sein du dataset (2016)")
##
####

fig.tight_layout()

plt.show()

rated["ENERGYSTARScore"].describe()


#### Observations :
- Les notes sont globalement hautes (Q1=53-Q3=89) - Les batiments ont le droit d'afficher le label "Energy Star Certified" si leur note est superieure ou égale a 75 -> Mediane (sources), motivation pour chercher a atteindre ce score.
- Variance haute (26).

### 2.b : Paire 2 : Note Energy Star en fonction de l'energie par metre carré.

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.barplot(
    x=rated["ENERGYSTARScore"].values.astype(int),
    y=rated["SourceEUI(kWh/m2)"].values.astype(float),
    ci=10
)

x_line = np.arange(0, 101, 5)

y_line = []

for x in x_line:
    ratings_x = rated[rated["ENERGYSTARScore"] == x]
    mean_x = ratings_x["SourceEUI(kWh/m2)"].mean(numeric_only=None)
    if pd.isna(mean_x):
        mean_x = rated["ENERGYSTARScore"].quantile(0.25).mean()
    y_line.append(mean_x)

ax1.plot(x_line, y_line)

###
# Titles/Lables
ax1.set_xticks(np.arange(-1, 101, 5))
ax1.set_ylabel("SourceEUI(kWh/m2)")
ax1.set_xlabel("Energy Star Score (0-100)")
fig.suptitle("Visualisation de 'l'energy use intensity' en fonction de la note E* des batiments")
#
###

fig.tight_layout()
plt.show()


#### Observation 
- On peut observer ici un lien clair entre le Score Energy Star (E*) et l'intensité d'utilisation énergétique (EUI). Le lien est logique dans ce cas : le site E* explique que, entre autres, l'utilisation d'énergie est un facteur.
- Le site internet d'E* fourni également une motivation pour l'obtention de cette note : un batiment est "Energy Star Certified" lorsque sa note est supérieure ou égale à 75 (ce qui peut expliquer le pic autour de cette note : les developpeurs immobiliers ont interèt à obtenir cette certification)

### 2.c Paire 3 : Rapport entre E* et année de construction :

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
)

years_list = list(df_seattle[df_seattle["YearBuilt"].notna()]["YearBuilt"].values)
year_keys = []
for year in years_list:
    if year not in year_keys:
        year_keys.append(year)

year_keys.sort()

year_dict = dict.fromkeys(year_keys)

for key in year_dict.keys():
    temp = df_seattle[(df_seattle["YearBuilt"] == key) & (df_seattle["ENERGYSTARScore"].notna())]
    scores = [int(i) for i in temp["ENERGYSTARScore"]]
    try:
        mean = sum(scores) / len(scores)
        year_dict[key] = mean
    except ZeroDivisionError:
        year_dict[key] = 0

ax1.bar(
    x=list(year_dict.keys()),
    height=list(year_dict.values()),
    color=used_cmap.colors,
    width=1,
    )

###
# Titles/Lables
ax1.set_yticks(np.arange(0, 100, 5))
ax1.set_ylabel("Note Energie Star Moyenne")
ax1.set_xlabel("Année de construction (1900, 2015)")
fig.suptitle("Représentation de la moyenne E* des batiments en fonction de leur année de construction")
#
###

fig.tight_layout()
plt.show()


#### Observation :
- L'année de construction ne semble pas de lien clair avec la note E* - Cela peut venir, comme ecrit plus haut, de travaux de modernisation par exemple.

### 2.d Paire 4 : Energy Star en fonction de l'émission de gazs a effets de serre.

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
)

data_ghg = df_seattle[
        (
            df_seattle["ENERGYSTARScore"].notna() &
            df_seattle["GHGEmissionsIntensity(kgCO2e/ft2)"].notna()
        )
    ]

ax1 = sns.barplot(
    x="ENERGYSTARScore",
    y="GHGEmissionsIntensity(kgCO2e/ft2)",
    data=data_ghg,
    ci=5
)

###
# Titles/Lables
ax1.set_xticks(np.arange(-1, 101, 5))
fig.suptitle("Moyenne E* en fonction de l'intensité d'emission de GES")
#
###
fig.tight_layout()
plt.show()


#### Observations :
- Meme si les données semblent varier enormément, il semblerait que les batiments obtenant de meilleurs scores soient en moyenne ceux qui relachent le moins de GES. Cela est moins flagrant que dans le cas de l'intensité d'utilisation electrique mais cette variable a son importance

## 3 : Représentation Property Type/Utilisation Energie

<i>On cherche s'il existe des correllations entre les principaux types de batiments et leut note E*</i>

### 3.a Visualisation des categories principales

In [ ]:
uniques = df_seattle["PrimaryPropertyType"].unique()
unique_ptype_dict = dict.fromkeys(uniques)
for unique in unique_ptype_dict.keys():
    unique_ptype_dict[unique] = len(df_seattle[df_seattle["PrimaryPropertyType"] == unique])

unique_ptype_dict


#### Observations :
- SPS -> Seattle Public School
- K12 -> Shool -> from kindergarten to 12th grade
- Typos, doubles espaces etc.
- Pré groupement via généralisation
- La generalisation permettra de creer des categories plus inclusives tout en gardant les propriétes principales des batiments
- Necessité de clarification : Batiments types cités universitaires & campus comptent comme residentiels ?

In [ ]:
correction_dict = {
    "Self-Storage Facility\n": "Self-Storage Facility",  # Possibly warehouse
    "Small- and Mid-Sized Office": "Small & Mid-Sized Office",
    "Restaurant\n": "Restaurant",
    "Distribution Center\n": "Supermarket & Grocery store",
    "Distribution Center": "Supermarket & Grocery store",
    "Supermarket/Grocery Store": "Supermarket & Grocery store",
    "Supermarket / Grocery Store": "Supermarket & Grocery store",
    "SPS-District K-12": "K-12 School",
    "Senior Care Community": "Small & Medium medical facility",  # Unsure
    "Laboratory": "Small & Medium medical facility",  # Unsure
    "Medical Office": "Small & Medium medical facility",  # Unsure
    "Small- and Mid-Sized Office": "Small & Mid-Sized Office",
    "Non-Refrigerated Warehouse": "Warehouse",
    "Residence Hall/Dormitory": "Residence Hall",
    "University": "College/University",
    "Retail Store": "Supermarket & Grocery store",
    "Self-Storage Facility": "Warehouse",  # Unsure
}


In [ ]:
for key in correction_dict.keys():
    df_seattle["PrimaryPropertyType"].replace(to_replace=key, value=correction_dict[key], inplace=True)

print(df_seattle["PrimaryPropertyType"].value_counts())



In [ ]:
select_six = [
    "Small & Mid-Sized Office", "Large Office", "Warehouse", "Supermarket & Grocery store",
    "K-12 School", "Small & Medium medical facility",
]


In [ ]:
# EUI / E* , color code property type, sample based on smallest group size

fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 16),
    dpi=pc_dpi,
)

data_ptype = df_seattle[df_seattle["PrimaryPropertyType"].isin(select_six)]

ax1 = sns.boxplot(
    x=data_ptype["PrimaryPropertyType"],
    y=data_ptype["SourceEUI(kWh/m2)"],
    data=data_ptype
)

###
# Titles/Lables
ax1.set_xticklabels(
    labels=select_six,
    rotation=45
    )
ax1.set_ylim(0, 1500)  # few outliers above
fig.suptitle("Efficacité d'utilisation energetique en fonction de l'utilisation principale des principaux groupes de batiments")

#
###

plt.tight_layout()
plt.show()


#### Observations :

- Small / Mid sized office EUI < Large Office EUI : Influence de la taille globale sur l'efficacite energetique ?

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 16),
    dpi=pc_dpi,
)

ax1 = sns.scatterplot(
    x="SiteEnergyUse(kWh)",
    y="PropertyArea(SquareMetre)Total",
    hue="PrimaryPropertyType",
    data=data_ptype
)

###
# Titles/Lables
ticks_y = ax1.get_yticks()
ax1.set_yticks([int(tick) for tick in ticks_y])
ticks_x = ax1.get_xticks()
ax1.set_xticklabels(ticks_x)
fig.suptitle("Utilisation d'energie en fonction de la taille des batiments, nuancés par leur utilisation principale")
#
###

plt.tight_layout()
plt.show()


### 3.b : Etude en détails de la correllation taille -> EUI

- Les boxplots ci dessus semblent montrer une relation de corrélation entre l'augmentation de l'efficacité d'utilisation énergétique et l'augmentation de la taille du meme "grand" type de batiment (small & mid sized office / large office)
- On peut étudier cette paire à par pour :
  - Vérifier s'il existe bel et bien une corrélation entre augmentation de taille et augmentation de l'EUI
  - Vérifier si la même hypothèse peut être appliquee avec la note E* (l'augmentation de l'intensité d'utilisation energetique entraine une baisse de la note E*, cf. 2.b), on s'attendrait ici, à une corrélation négative entre les variables

#### 3.b.1 : Taille / EUI

- Les batiments de type small and mid sized office semblent avoir une intensité d'utilisation energetique plus faible que les batiments de la catégorie Large Office

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 16),
    dpi=pc_dpi,
)

office_ptypes = ["Small & Mid-Sized Office", "Large Office"]

data_ptype_off = df_seattle[df_seattle["PrimaryPropertyType"].isin(office_ptypes)]

ax1 = sns.boxplot(
    x=data_ptype_off["PrimaryPropertyType"],
    y=data_ptype_off["SourceEUI(kWh/m2)"],
    data=data_ptype_off
)

###
# Titles/Lables
ax1.set_xticklabels(
    labels=office_ptypes,
    rotation=45
    )
fig.suptitle("Efficacité d'utilisation energetique en fonction de l'utilisation principale des principaux groupes de batiments |-> Bureaux uniquements")
#
###

plt.tight_layout()
plt.show()

print(
    "Small & Mid-Sized Office : ",
    data_ptype_off[data_ptype_off["PrimaryPropertyType"] == "Small & Mid-Sized Office"]["SourceEUI(kWh/m2)"].describe()
    )
print("#######################################")
print(
    "Large Office : ",
    data_ptype_off[data_ptype_off["PrimaryPropertyType"] == "Large Office"]["SourceEUI(kWh/m2)"].describe()
    )


##### <u> Observation : </u>
- Dans ce cas les données semblent être similaires, ce qui n'etait pas le cas dans le boxplot regroupant les 6 principales categories
- Utilisation du Student's T-Test pour verifier si la difference est significative :
  - Les conditions préalables du T-Test, comme pour l'ANOVA sont :
    - Homogeneite de la variance -> Test de Bartlett
    - Residus gaussiens -> Test de Shapiro-Wilks
    - Données independantes -> C'est le cas dans cette étude
    - on definit alpha à 5%
  - On formule l'hypothèse nulle H0 telle que : H0 = Les differences entre l'Intensité d'utilisation energétique ne sont pas dus à la taille des batiments de bureaux

In [ ]:
alpha = 0.05


In [ ]:
large_office_eui = df_seattle[
        (df_seattle["PrimaryPropertyType"] == "Large Office") &
        (df_seattle["SourceEUI(kWh/m2)"].notna())
    ]["SourceEUI(kWh/m2)"]

small_m_office_eui = df_seattle[
        (df_seattle["PrimaryPropertyType"] == "Small & Mid-Sized Office") &
        (df_seattle["SourceEUI(kWh/m2)"].notna())
    ]["SourceEUI(kWh/m2)"]

ttest_office = stats.ttest_ind(a=small_m_office_eui, b=large_office_eui, equal_var=False)

print(ttest_office, "\n", f"p-values > alpha : {ttest_office.pvalue > alpha}")


In [ ]:
# Null hypothesis : homoscedasticity

bartlett_test_office = stats.bartlett(large_office_eui, small_m_office_eui)

print(bartlett_test_office, "\n", f"p-values > alpha : {bartlett_test_office.pvalue > alpha}")


#### Analyse du t-test 1 --> EUI :
- Le test de Bartlett ne permet pas d'accepter l'hypothèse alternative telle que : la variance inter echantillon n'est pas homogène, on accepte l'hypothèse nulle : les variances des echantillons sont homogènes
- Le test de Student / t-test ne permet de refuter l'hypothèse nulle telle que : H0 = L'augmentation de l'EUI n'a rien à voir avec la difference de taille des immeubles de bureau.
- Une analyse des données brutes, avant l'élimination des valeurs aberrantes via IQR, montrait que, même si la difference d'EUI n'etait pas statistiquement significative, la note E* variait, elle, significativement en fonction de la taille des batiments
- On realise les memes tests mais ici sur la note E*

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 16),
    dpi=pc_dpi,
)

office_ptypes = ["Small & Mid-Sized Office", "Large Office"]

data_ptype_off = df_seattle[df_seattle["PrimaryPropertyType"].isin(office_ptypes)]

ax1 = sns.boxplot(
    x=data_ptype_off["PrimaryPropertyType"],
    y=data_ptype_off["ENERGYSTARScore"],
    data=data_ptype_off
)

###
# Titles/Lables
ax1.set_xticklabels(
    labels=office_ptypes,
    rotation=45
    )
fig.suptitle("Efficacité d'utilisation energetique en fonction de l'utilisation principale des principaux groupes de batiments |-> Bureaux uniquements")
#
###

plt.tight_layout()
plt.show()

print(
    "Small & Mid-Sized Office : ",
    data_ptype_off[data_ptype_off["PrimaryPropertyType"] == "Small & Mid-Sized Office"]["ENERGYSTARScore"].describe()
    )
print("#######################################")
print(
    "Large Office : ",
    data_ptype_off[data_ptype_off["PrimaryPropertyType"] == "Large Office"]["ENERGYSTARScore"].describe()
    )


#### T-Test 2 : E*/Taille

- Memes critères qu'énoncés plus haut
- Hypothèse nulle H0 : La taille des batiments n'a pas d'influence significative sur leur note Energy Star

In [ ]:
large_off_estar = df_seattle[
        (df_seattle["PrimaryPropertyType"] == "Large Office") &
        (df_seattle["ENERGYSTARScore"].notna())
    ]["ENERGYSTARScore"].astype(int)

small_med_off_estar = df_seattle[
        (df_seattle["PrimaryPropertyType"] == "Small & Mid-Sized Office") & 
        (df_seattle["ENERGYSTARScore"].notna())
    ]["ENERGYSTARScore"].astype(int)


In [ ]:
# Null hypothesis : homoscedasticity

bartlett_test_estar_off = stats.bartlett(large_off_estar, small_med_off_estar)

print(bartlett_test_estar_off, "\n", f"p-values > alpha : {bartlett_test_estar_off.pvalue > alpha}")


In [ ]:
# Std not normally distributed with alpha @5%, using ttest ind with parameter : equal_var = False

ttest_estar_off = stats.ttest_ind(a=large_off_estar, b=small_med_off_estar, equal_var=False)

print(ttest_estar_off, "\n", f"p-values < alpha : {ttest_estar_off.pvalue < alpha}")


### Analyse du t-test 2 --> Bureaux/E* : 
- Le test de Bartlett ne permet pas de refuter l'hypothèse nulle telle que : la variance inter echantillon n'est pas homogène. On ajuste les parametres --> `equal_var=False`
- Le test de Student / t-test permet de refuter l'hypothèse nulle telle que : H0 = Le baisse globale des notes n'est pas corrélée avec la taille des bureaux. On accepte l'hypothèse alternative H1, à savoir : Il existe une corrélation négative entre la taille d'un immeuble de bureau et sa note E*

#### Il est egalement possible d'effectuer une ANOVA pour elargir le spectre de l'etude a toutes les categories

### 4. Reduction de dimension pour alléger le dataset en vue d'exportation pour modeles predictifs

Pour rendre lisible les modelisations predictives, il est necessaire de reduire la dimension du dataset pour que le moins d'attributs possibles puissent fournir le maximum d'information.
- Dans un premier temps, il est envisageable de realiser une reduction de dimension sur les variables, si possible via ACP (si cette approche se révèle trop peu precise, les methodes non lineaires telles que T-SNE ou UMAP sont envisageables)
- Dans un second temps, l'étude ci dessus a pu montrer que les batiments appartenant aux memes "categories" partagent des caracteristiques et des trends qui peuvent etre exploitables pour traiter les objets en tant que clusters plutot que separement (K-Means)

#### 4.a : Elimination des variables inutiles la modelisation

- Certaines variables sont inutiles a la modelisation. Si elles permettent de mieux comprendre la note E*, un grand nombre de variable sera inutile a la creation de modeles
- Dans un premier temps : filtrage manuel des features
- Creation d'une copie du df_seattle - il regroupe des features utiles a d'autres analyses (df_export)
- On peut "Eliminer" la variable `OSEBuildingID` en la considerant comme l'index du dataset (unique id)

In [ ]:
df_export = df_seattle

print(df_export.columns)


In [ ]:
manual_suppression_list = [
    "PropertyArea(SquareMetre)Parking", "PropertyArea(SquareMetre)Building(s)", # row 1 & 2.1 = duplicates/parts of PropertyAreaTotal
    "LargestPropertyUseTypeArea(SquareMetre)", "PropertyName", "BuildingType",
    "NumberofBuildings", "NumberofFloors",
    ]

df_export.drop(columns=manual_suppression_list, inplace=True, errors="ignore")
df_export.set_index("OSEBuildingID", inplace=True)



In [ ]:
df_export.columns


#### 4.a : Realisation d'une ACP sur les colonnes restantes

In [ ]:
ignore_pca = [
    "Neighborhood", "PrimaryPropertyType", "LargestPropertyUseType",  # Usable features in Kmeans but not relevant here
    "ENERGYSTARScore" # Baseline
]

columns_pca = [col for col in df_export.columns if col not in ignore_pca]

print(columns_pca)


In [ ]:

data_pca = df_export[columns_pca]

data_pca = data_pca.astype(float)  # All values are forced converted to float, as the columns above (columns_pca) are all numeric

data_pca = data_pca.fillna(data_pca.mean())

X = data_pca.values
names = df_export.index
features = columns_pca

scaled_data = preprocessing.scale(data_pca)
pca = PCA(n_components=len(columns_pca))
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)
percentage_variation = np.round(pca.explained_variance_ratio_ * 100, decimals=1)

pcs = pca.components_


##### 4.a.1 : Eboulis des valeurs propres | Scree plot :

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(24, 10),
    dpi=pc_dpi,
)

x_bars = np.arange(1, len(percentage_variation) + 1, 1)

cummulative_percentage = np.cumsum(percentage_variation)

ax1.bar(x_bars, height=percentage_variation)  # Inertia Scree

###
# Titles/Lables
labels = ["PC" + str(c) for c in range (1, len(percentage_variation) + 1)]
ax1.set_xticks(range(1, len(labels) + 1, 1))
ax1.set_xticklabels(labels)
ax1.set_xlabel("Rang des composants principaux")
ax1.set_ylabel("Pourcentage d'inertie")
fig.suptitle("Eboulis des valeurs propres (histogramme), pourcentage d'inertie cummulée (ligne rouge)")
ax1.text(2, 90, 'Black line : 80% marker')
#
###

ax1.plot(ax1.get_xticks(), cummulative_percentage, marker="o", color="r", linewidth=1)  # Cumulative Inertia
ax1.plot(np.arange(0, len(cummulative_percentage) + 1, 1), [80 for _ in range(0, len(cummulative_percentage) + 1)], color="0", linewidth=2.5)
fig.tight_layout()

plt.show()


##### Observations : 
- PC1 -> PC3 regroupent 80+% de l'intertie totale, on peut également inclure PC4 si necessaire
- Etude des corrélations des Principaux Composants

##### 4.a.2 : Cercles de corrélation et dataframe details :

In [ ]:
pc_couples = [(0, 1), (2, 3)]

for couple in pc_couples:
    display_circles(pca=pca, pca_cols=columns_pca, couple_pc=couple)


In [ ]:
pca_explained_df = show_contribution(pca=pca, columns_pca=columns_pca, lim_pc=4)

pca_explained_df


##### 4.a.3 Details des coefficients de correlation des composants principaux
- On cherche a exprimer les valeurs telles que , pour f1 --> fn , PC = *a x f1 + b x f2 + ... + m x fn*
- Utilisation des valeurs absolues

In [ ]:
pc_one = pca_explained_df["PC1"]
pc_one.sort_values(ascending=False)


In [ ]:
pc_two = pca_explained_df["PC2"]
print(pc_two, "\n")

print("##############################################")

pc_two.abs().sort_values(ascending=False)


In [ ]:
pc_three = pca_explained_df["PC3"]
print(pc_three, "\n")

print("##############################################")

pc_three.abs().sort_values(ascending=False)


In [ ]:
pc_four = pca_explained_df["PC4"]
print(pc_four, "\n")

print("##############################################")

pc_four.abs().sort_values(ascending=False)


#### ACP : 

- PC1 : <b>46.9%</b> Explained Variance, Majoritairement composé de l'utilisation electrique et l'efficacité (Consommation/Metre Carré) et des composants de ces statistiques :
  - SiteEnergyUse(kWh)                    0.448471
  - SiteEnergyUseWN(kWh)                  0.446842
  - Electricity(kWh)                      0.389923
  - PropertyArea(SquareMetre)Total        0.313987
  - SourceEUI(kWh/m2)                     0.301655
  - SourceEUIWN(kWh/m2)                   0.300089
<hr>

- PC2 : <b>20.6%</b> Explained Variance, Plus gros coefficients liés à l'émission de pollution, anticorrélé avec l'utilisation electrique : 
  - GHGEmissionsIntensity(kgCO2e/ft2)     0.559182
  - NaturalGas(kWh)                       0.510974
  - GHGEmissions(MetricTonsCO2e)          0.421606  --> Plus polluant que l'electricité
  - Electricity(kWh)                     -0.319903

<hr>

- PC3 : <b>16.1%</b> Explained Variance, très sembable à PC1 (valeurs absolues)

<hr>

- PC4 : <b>9%</b> Explained Variance, pratiquement entierement basé sur l'age des batiments

<hr>

Un modele predictif utilisant comme variables principales PC1 (utilisation electrique) et PC2 (Pollution 'directe'), avec pour baseline l'E* peut être une piste interessante.


In [ ]:
display_circles(pca=pca, pca_cols=columns_pca, couple_pc=(0, 2))


##### Verification : 
- Les fleches du cercle de corrélation PC1/PC3 ont un angle proche de 45 degrés / 135 degrés, indicant une similitude des deux composants principaux en se basant sur les valeurs absolues


In [ ]:
pca_prep = data_pca


In [ ]:
columns = [f"PC{idx + 1}" for idx in range(pca.n_components_)]
df_pca = pd.DataFrame(pca.transform(pca_prep), columns=columns, index=pca_prep.index)

df_pca.head()


In [ ]:
## Removing cols from PC3 to PC 10

df_pca.drop(columns=[f"PC{idx}" for idx in range(3, 11)], inplace=True)


In [ ]:
# Renaming PC1 to Energy_consumption and PC2 to Pollution_generated

df_pca.rename(columns={
        "PC1": "Energy_consumption_(PC1)",
        "PC2": "Pollution_generated_(PC2)"
        },
        inplace=True
    )


In [ ]:
# Adding Baseline and other infos

additionnal_infos = df_export[
        [
        "SourceEUI(kWh/m2)", "GHGEmissionsIntensity(kgCO2e/ft2)", "ENERGYSTARScore"
        ]
    ]

df_pca = df_pca.join(additionnal_infos, how="outer")


In [ ]:
print(df_pca.shape)
df_pca.head()


## Conclusion :

1. Nous avons identifier les variables importantes et les synthetiser en composants principaux représentant à eux deux 67.5% de la variance et couvrant les features les plus importantes.
2. L'analyse a révélé que des features comme le type de batiment pouvaient avoir une influence sur la note E*
3. L'année est exportée à titre indicatif et parce que PC4 semblait fortement et majoritairement corrélé a cette variable (9% de la variance)
4. La variable PC1 regroupe les variables les plus importantes pour le calcul de l'efficacité énergétique
5. La variable PC2 représente la pollution généré par un batiment en grande majorité
6. Les composants principaux ont deja été mis à l'echelle lors de l'ACP. Leur utilisation dans des algorithmes de machine learning ne causera pas de probleme de sur(sous)-représentation
7. Sur plus de 3300 batiments initiaux, 868 ont été retenus

#### Scaling des donnees entre elles, preservation de l'E* score

#### Etude de la pertinence des deux composants principaux selectionnés

- -> 1 Projection de PC1/E*
- -> 2 Projection de PC2/E*

In [ ]:
df_pca = scale_df(dataframe_to_scale=df_pca, constant_col="ENERGYSTARScore")


In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.lineplot(
    x="ENERGYSTARScore",
    y="Energy_consumption_(PC1)",
    data=df_pca,
    ci=20
)


###
# Titles/Lables
fig.suptitle("Representation PC1 moyen en fonction de note E*")
ax1.set_xticks(np.arange(0, 101, 5))
#
###

fig.tight_layout()
plt.show()


In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.lineplot(
    x="ENERGYSTARScore",
    y="Pollution_generated_(PC2)",
    data=df_pca,
    ci=20
)


###
# Titles/Lables
fig.suptitle("Representation PC2 moyen en fonction de note E*")
ax1.set_xticks(np.arange(0, 101, 5))
#
###

fig.tight_layout()
plt.show()


In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.lineplot(
    x="ENERGYSTARScore",
    y="SourceEUI(kWh/m2)",
    data=df_pca,
    ci=20
)


###
# Titles/Lables
ax1.set_xticks(np.arange(0, 101, 5))
fig.suptitle("Representation SourceEUI moyen en fonction de note E*")
#
###

fig.tight_layout()
plt.show()


In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.lineplot(
    x="ENERGYSTARScore",
    y="GHGEmissionsIntensity(kgCO2e/ft2)",
    data=df_pca,
    ci=20
)


###
# Titles/Lables
ax1.set_xticks(np.arange(0, 101, 5))
fig.suptitle("Representation d'emissions de GES moyen en fonction de note E*")
#
###

fig.tight_layout()
plt.show()


#### Les composants principaux ne permettent pas, à premiere vue, d'établir une corrélation entre eux et la note E*, utilité de l'ACP discutable

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.scatterplot(
    x="SourceEUI(kWh/m2)",
    y="Energy_consumption_(PC1)",
    data=df_pca,
    hue="ENERGYSTARScore",
    ci=20
)


###
# Titles/Lables
fig.suptitle("Representation SourceEUI en fonction du composant principal 1")
#
###

fig.tight_layout()
plt.show()


In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.scatterplot(
    x="GHGEmissionsIntensity(kgCO2e/ft2)",
    y="Pollution_generated_(PC2)",
    data=df_pca,
    hue="ENERGYSTARScore",
    ci=20
)


###
# Titles/Lables
fig.suptitle("Representation emission GES en fonction du composant principal 2")
#
###

fig.tight_layout()
plt.show()


In [ ]:
df_pca.to_csv("./data/seattle_scaled_clean_data.csv")
